<a href="https://colab.research.google.com/github/h679177/ml_project/blob/main/launch_fil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
pip install --upgrade gradio

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import joblib

path = '/content/drive/MyDrive/Colab_Notebooks'

loaded_model = joblib.load(path + '/trained_model_joblibvs.pkl')

In [3]:
def convert_binary_input(input):
  if input == "Yes":
    input = 1

  else:
    input = 0

  return input

In [4]:
def convert_parent_maxEdu(input):
  if input == 'Up to 4th grade':
    input = 1
  elif input == '5th to 9th grade':
    input = 2
  elif input == 'Secondary education':
    input = 3
  else:
    input = 4

  return input

In [5]:
def convert_studytime(input):
  if input == 'Less than 2 hours':
    input = 1
  elif input == '2-5 hours':
    input = 2
  elif input == '5-10 hours':
    input = 3
  else:
    input = 4

  return input

In [13]:
def convert_reason_reputation(input):
  if input == 'Reputation':
    input = 1
  else:
    input = 0
  return input

def convert_reason_course(input):
  if input == 'The courses it offers':
    input = 1
  else:
    input = 0

  return input

def convert_reason_other(input):
  if input == 'other':
    input = 1
  else:
    input = 0

  return input


def convert_reason_close(input):
    if input == 'It´s close to home':
      input = 1
    else:
      input = 0

    return input

In [7]:
def convert_travel(input):
  if input == 'Less than 15 minutes':
    input = 1
  elif input == '15-30 minutes':
    input = 2
  elif input == '30 mins - 1 hour':
    input = 3
  else:
    input = 4

  return input

In [8]:
import pandas as pd

In [24]:
import gradio as gr

def get_data(higher_education, failed, urban, absence, stay_at_home_parent, parent_teacher,
             parent_maxEducation, weekday_alcohol, weekend_alcohol, studytime, travel_time, reason_for_school,
             freeTime, goOut, family_relations):

    higher_education = convert_binary_input(higher_education)
    failed = convert_binary_input(failed)
    urban = convert_binary_input(urban)
    stay_at_home_parent = convert_binary_input(stay_at_home_parent)
    parent_teacher = convert_binary_input(parent_teacher)
    parent_maxEducation = convert_parent_maxEdu(parent_maxEducation)
    studytime = convert_studytime(studytime)
    travel_time = convert_travel(travel_time)
    reason_rep = convert_reason_reputation(reason_for_school)
    reason_other = convert_reason_other(reason_for_school)
    reason_course = convert_reason_course(reason_for_school)
    reason_home = convert_reason_close(reason_for_school)

    input_data = pd.DataFrame({
        'address' : [urban],
        'traveltime': [travel_time],
        'studytime' : [studytime],
        'failures' : [failed],
        'higher' : [higher_education],
        'famrel' : [family_relations],
        'freetime' : [freeTime],
        'goout' : [goOut],
        'Dalc' : [weekday_alcohol],
        'Walc' : [weekend_alcohol],
        'absences' : [absence],
        'reason_course' : [reason_course],
        'reason_home' : [reason_home],
        'reason_other' : [reason_other],
        'reason_reputation' : [reason_rep],
        'parent_maxEdu' : [parent_maxEducation],
        'parent_teacher' : [parent_teacher],
        'parent_at_home' : [stay_at_home_parent]
         })

    #predcition = loaded_model.predict(input_data)

    probability = loaded_model.predict_proba(input_data)[0, 1]
    threshold = 0.65
    predcition = 1 if probability >= threshold else 0

    if predcition == 1:
      output = 'It doesn´t look like you´re at risk of failing!'
    else:
      output = 'It looks like you might be at risk of failing!'

    return output

demo = gr.Interface(
    fn=get_data,
    inputs=[gr.Radio(choices=['Yes', 'No'], label="Do you want higher education?"),
            gr.Radio(choices=['Yes', 'No'], label="Have you failed any classes previously?"),
            gr.Radio(choices=['Yes', 'No'], label="Do you live urban?"),
            gr.Number(label = "How many days absence do you have?"),
            gr.Radio(choices=['Yes', 'No'], label = "Is one or more of your parent a stay at home parent?"),
            gr.Radio(choices=['Yes', 'No'], label="Is at least one of your parents a teacher?"),
            gr.Radio(choices=['Up to 4th grade', '5th to 9th grade', 'Secondary education', 'Higher education'], label="What is the highest level of education your parents have?"),
            gr.Slider(minimum=1, maximum=5, step=1, label="How much alcohol do you consume during the school week? (1 being very little and 5 being very much)"),
            gr.Slider(minimum=1, maximum=5, step=1, label="How much alchol do you consume during the weekend? (1 being very little and 5 being very much)"),
            gr.Radio(['Less than 2 hours', '2-5 hours', '5-10 hours', 'More than 10 hours'], label="Approximately how many hours do you study every week?"),
            gr.Radio(['Less than 15 minutes', '15-30 minutes', '30 mins - 1 hour', 'More than 1 hour'], label="Approximately how long do you have to travel to get to school?"),
            gr.Radio(['Reputation', 'The courses it offers', 'It´s close to home', 'Other'], label="What´s the reason you chose your current school?"),
            gr.Slider(minimum=1, maximum=5, step=1, label="How much free time do you have after school? (1 being very little and 5 being very much)"),
            gr.Slider(minimum=1, maximum=5, step=1, label="How often do you go out with friends? (1 being rarely and 5 being very often)"),
            gr.Slider(minimum=1, maximum=5, step=1, label="What´s your relationship with your family like? (1 being very bad and 5 being excellent)"),
            ],
    outputs=["text"],
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8f983f3b93933464f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
